# Basic Example
Make a basic likelihood (and posterior) example for *What is a Measurement?*

## Author:
- **David W. Hogg** (NYU) (MPIA) (Flatiron)

## Notes:
- This is part of the *What is a Measurement?* project. Copyright the author.

In [ ]:
import numpy as np
import pylab as plt
import scipy.optimize as op

In [ ]:
N = 16

def expectation(ts, pars):
    om, amp, phi, y0 = pars
    return y0 + amp * np.cos(om * ts + phi)

def make_fake_data(seed=17):
    rng = np.random.default_rng(seed)
    ts = np.sort(7. * rng.uniform(size=N))
    ivars = 1. + 4. * rng.uniform(size=N)
    omega = 0.5 + 3. * rng.uniform()
    amp = 1. + rng.uniform()
    phi = 2. * np.pi * rng.uniform()
    y0 = 2. + 2. * rng.uniform()
    truepars = np.array((omega, amp, phi, y0))
    return ts, expectation(ts, truepars) + rng.normal(size=N) / np.sqrt(ivars), ivars, truepars

In [ ]:
ts, ys, ivars, true_pars = make_fake_data()
print(ts.shape, ys.shape, true_pars)

In [ ]:
def plot(ts, ys, ivars, true_pars, ml_pars, title):
    plt.errorbar(ts, ys, yerr=1./np.sqrt(ivars), fmt="ko")
    plot_ts = np.linspace(0., 7., 1000)
    if true_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, true_pars), "r-", lw=1, alpha=0.45)
    if ml_pars is not None:
        plt.plot(plot_ts, expectation(plot_ts, ml_pars), "r-", lw=2, alpha=0.9)
    plt.xlabel("time")
    plt.ylabel("data value")
    plt.title(title)

plot(ts, ys, ivars, true_pars, None, "data and true expectation")

In [ ]:
def negative_log_likelihood(pars, ts, ys, ivars):
    return 0.5 * np.sum(ivars * (ys - expectation(ts, pars)) ** 2)

In [ ]:
res = op.minimize(negative_log_likelihood, true_pars, args=(ts, ys, ivars))
print(res)
ml_pars = np.zeros(4) + np.nan
if res.success:
    ml_pars = res.x
print(ml_pars)

In [ ]:
plot(ts, ys, ivars, true_pars, ml_pars,
     "data, true expectation, and maximum-likelihood expectation")

In [ ]:
def negative_partial_log_likelihood(al, om, ts, ys, ivars):
    pars = np.append(om, al)
    return 0.5 * np.sum(ivars * (ys - expectation(ts, pars)) ** 2)

def negative_profile_log_likelihood(om, ts, ys, ivars, guess):
    res = op.minimize(negative_partial_log_likelihood, guess, args=(om, ts, ys, ivars))
    if res.success:
        return res.fun
    return np.nan

In [ ]:
oms = np.linspace(0.5, 3.5, 100)
plls = np.array([negative_profile_log_likelihood(om, ts, ys, ivars, true_pars[1:]) for om in oms])
print(plls.shape)

In [ ]:
plt.plot(oms, -1. * plls, "k-")
plt.axvline(true_pars[0], c="r", lw=1, alpha=0.45)
plt.axvline(ml_pars[0], c="r", lw=2, alpha=0.9)
plt.xlabel("angular frequency")
plt.ylabel("profile log likelihood")

In [ ]:
n_trials = 128
many_true_pars = np.zeros((n_trials, 4)) + np.nan
many_ml_pars = np.zeros((n_trials, 4)) + np.nan
for trial in range(n_trials):
    ts, ys, ivars, tp = make_fake_data(seed=trial)
    many_true_pars[trial] = tp
    res = op.minimize(negative_log_likelihood, tp, args=(ts, ys, ivars))
    if res.success:
        many_ml_pars[trial] = res.x
print(np.sum(np.isnan(many_ml_pars[:, 0])))

In [ ]:
plt.plot([-10, 10], [-10, 10], "k-", alpha=0.45)
plt.scatter(many_true_pars[:,0], many_ml_pars[:,0], s=3, c="k", alpha=0.9)
plt.axis("equal")
plt.xlim(0.5, 3.5)
plt.ylim(0.5, 3.5)
plt.xlabel("true angular frequency")
plt.ylabel("maximum-likelihood angular frequency")